In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.impute import SimpleImputer


In [ ]:
os.makedirs('data', exist_ok=True)

PATH = 'data/pakwheels_used_car_data_v02.csv'
if not os.path.exists(PATH):
    raise FileNotFoundError(PATH)

df = pd.read_csv(PATH)


In [ ]:
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]


In [ ]:
keep = [
    'make','model','year','engine','transmission','fuel',
    'mileage','city','assembly','body','color',
    'registered','price'
]

cols = [c for c in keep if c in df.columns]
df = df[cols].copy()


In [ ]:
for c in ['price','year','engine','mileage']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')


In [ ]:
df = df.dropna(subset=['price'])


In [ ]:
if 'make' in df.columns:
    for c in ['year','engine','mileage']:
        if c in df.columns:
            df[c] = df.groupby('make')[c].transform(
                lambda x: x.fillna(x.median())
            )


In [ ]:
for c in ['year','engine','mileage']:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())


In [ ]:
for c in ['make','model','transmission','fuel','city',
          'assembly','body','color','registered']:
    if c in df.columns:
        df[c] = (
            df[c]
            .astype(str)
            .str.strip()
            .str.title()
            .replace({'Nan':'Unknown','None':'Unknown'})
        )
        if df[c].isnull().any():
            df[c] = df[c].fillna('Unknown')


In [ ]:
df = df.drop_duplicates()


In [ ]:
now = datetime.now().year

if 'year' in df.columns:
    df['car_age'] = now - df['year']
    df.loc[df['car_age'] < 0, 'car_age'] = np.nan


In [ ]:
num_cols = ['price','mileage','engine','year','car_age']
num_cols = [c for c in num_cols if c in df.columns]

for c in num_cols:
    q01 = df[c].quantile(0.01)
    q99 = df[c].quantile(0.99)
    df[c] = df[c].clip(lower=q01, upper=q99)


In [ ]:
bounds = {
    'price': (50000, 50000000)
}

if 'mileage' in df.columns:
    bounds['mileage'] = (0, 500000)

if 'engine' in df.columns:
    bounds['engine'] = (500, 5000)

if 'year' in df.columns:
    bounds['year'] = (1990, now)

for col, (lo, hi) in bounds.items():
    if col in df.columns:
        df = df[(df[col] >= lo) & (df[col] <= hi)]


In [ ]:
if 'mileage' in df.columns and 'car_age' in df.columns:
    df['mileage_per_year'] = df['mileage'] / (df['car_age'] + 1)


In [ ]:
if 'engine' in df.columns:
    bins = [0, 1000, 1500, 2000, 3000, 5000]
    labels = ['Small','Medium','Large','V-Large','X-Large']
    df['engine_category'] = pd.cut(df['engine'], bins=bins, labels=labels)


In [ ]:
age_bins = [0, 3, 7, 15, 100]
age_labels = ['New','Fairly Used','Used','Old']

if 'car_age' in df.columns:
    df['age_category'] = pd.cut(df['car_age'], bins=age_bins, labels=age_labels)


In [ ]:
luxury = ['Mercedes','Bmw','Audi','Lexus','Land Rover']
economy = ['Suzuki','Daihatsu','Proton','Faw']

if 'make' in df.columns:
    df['brand_category'] = df['make'].apply(
        lambda x: 'Luxury' if x in luxury 
        else ('Economy' if x in economy else 'Standard')
    )


In [ ]:
if 'price' in df.columns and 'engine' in df.columns:
    df['price_per_cc'] = df['price'] / df['engine']


In [ ]:
cleaned_path = 'data/cleaned_car_data.csv'
df.to_csv(cleaned_path, index=False)

print('cleaned saved', cleaned_path)
